In [4]:
'''
Created By: Adam Warren, Ankit Butola
Created Date: 10/30/2023
'''

import tkinter
from tkinter import filedialog, messagebox
from tkinter import ttk
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
import openpyxl
from folium.plugins import HeatMap, MarkerCluster
from os import path

# intiate global variables
excel_path = None
data = None # this will hold the excel workfile
shapefile_low_res = None # this will hold the shapefile at a low resolution
shapefile_high_res = None # this will hold the shapefile at a high resolution
country_name = None # this will hold the variable that names the country 
map_output = None # this will hold the map output wanted 
file_name = None # this will hold the file name they want to save it as 
file_path = None

def get_excel_doc():
    global data, excel_path
    root = tkinter.Tk()
    excel_doc = filedialog.askopenfilename()
    excel_path = excel_doc
    root.withdraw()
    if excel_doc.split(".")[-1] in ["xlsx", "xls"]:
        data_1 = pd.read_excel(excel_doc)
        data_1.columns = ['ID', 'latitude', 'longitude']
        data = data_1
    else:
        messagebox.showerror(title="file type error", message="you chose the wrong file type, try again.")
        
def choose_country():
    global country_name, progress_value, shapefile_low_res, shapefile_high_res
    selected_indices = listbox.curselection()
    if selected_indices:
        index = selected_indices[0]  # Assuming you expect only one selection
        country_name = listbox.get(index)
    base_path = r'J:\cms\External\TeamMembers\Adam Warren'
    if country_name == 'Saint Vincent and the Grenadines':
        # choose the mid res and the high res for the two shapefiles
        shapefile_low_res = rf'J:\cms\External\TeamMembers\Adam Warren\LAC COUNTRY SHAPEFILES - DO NOT MOVE\2. World LowRes\Saint Vincent and the Grenadines\Saint Vincent and the Grenadines.shp'
    else:
        shapefile_low_res = rf'J:\cms\External\TeamMembers\Adam Warren\LAC COUNTRY SHAPEFILES - DO NOT MOVE\2. World LowRes\{country_name}\{country_name}.shp'
    shapefile_high_res = rf'J:\cms\External\TeamMembers\Adam Warren\LAC COUNTRY SHAPEFILES - DO NOT MOVE\5. World HighRes\{country_name}\{country_name}.shp'

def choose_output():
    global map_output
    selected_indices = listbox2.curselection()
    if selected_indices:
        index = selected_indices[0]  # Assuming you expect only one selection
        map_output = listbox2.get(index)
        
def get_file_name():
    global file_name, file_path
    text = textentry_file.get()
    file_name = text
    file_x = excel_path.replace(excel_path.split("/")[-1], "")
    file_path = file_x + str(file_name) + '.html'
    
def finished_tasks():
    global file_path, start_time
    if path.exists(file_path):
        messagebox.showinfo(title="task completed", message="All tasks are completed, you can now close all windows")
        
def plot_maps(data, country_geometry_high_res, map_output, inside, outside_high_res, inside_count, file_path, country_geometry_low_res):    
    
    map_plot = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_starts=4, tiles='CartoDB positron')
    country_geojson = country_geometry_high_res.__geo_interface__
    folium.GeoJson(country_geojson, name='Country Boundary',style_function=lambda feature:{
                        'fillColor': 'lightyellow',
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.3,      
                        }
                      ).add_to(map_plot)
    
    heat_data = [[row['latitude'], row['longitude']] for index, row in data.iterrows() if Point(row['longitude'], row['latitude']).within(country_geometry_low_res)]
    marker_cluster = folium.plugins.MarkerCluster(name="Coordinate Points", options={'disableClusteringAtZoom':10},show=False).add_to(map_plot)

    #overlay group
    overlay_group = folium.FeatureGroup(name='Overlay Layers',show=False)
    overlay_group.add_to(map_plot)

    if map_output == 'Heatmap':
        HeatMap(heat_data,name="HeatMap", show = False).add_to(map_plot)

    elif map_output == 'Heatmap Overlay':
        HeatMap(heat_data,name="HeatMap", show = False).add_to(map_plot)
        for lat_long,count in inside_count.items():
            folium.Circle(lat_long,popup={lat_long},
                            radius=count*5, color ='black', fill=True,fill_opacity=0.5).add_to(overlay_group)

    elif map_output == 'Point Data Map':
        for lat_long,count in inside_count.items():
            folium.Marker(lat_long,popup=f"Inside Boundaries<br>{lat_long}",
                            icon=folium.Icon(color='green')).add_to(marker_cluster)
            folium.Circle(lat_long,popup={lat_long},
                            radius=count*5, color ='black', fill=True, fill_opacity=0.5).add_to(overlay_group)
    else:
        HeatMap(heat_data, name="HeatMap", show = False).add_to(map_plot)
        for lat_long,count in inside_count.items():
            folium.Marker(lat_long,popup=f"Inside Boundaries<br>{lat_long}",
                            icon=folium.Icon(color='green')).add_to(marker_cluster)
            folium.Circle(lat_long,popup={lat_long},
                            radius=count*5, color ='black', fill=True, fill_opacity=0.5).add_to(overlay_group)

    for ID,lat,lon in outside_high_res:
        folium.Marker(location=[lat,lon], popup=f"Outside Boundaries<br>ID:{ID}<br>{lat}<br>{lon}",
                            icon=folium.Icon(color='red')).add_to(map_plot)

    # Adding legend for differentiating Inside and Outside coordinate points
    legend_html = """
        <div style="position: fixed; bottom:30px; right:30px; z-index:9999; font-size:11px;">
            <p><i class="fa fa-circle fa-1x" style="color:green"></i> Inside Boundary</p>
            <p><i class="fa fa-circle fa-1x" style="color:red"></i> Outside Boundary</p>
        </div>
        """
    map_plot.get_root().html.add_child(folium.Element(legend_html))

    # Adding various tiles option incoorporated within Folium library
    folium.TileLayer('CartoDB positron').add_to(map_plot) #Selected as Default
    folium.TileLayer('OpenStreetMap').add_to(map_plot) 
    folium.TileLayer('Stamen Terrain').add_to(map_plot)
    folium.TileLayer('Stamen Toner').add_to(map_plot)
    folium.TileLayer('esrinatgeoworldmap', name='Esri NatGeo WorldMap', attr=' Esri, Delorme, NAVTEQ').add_to(map_plot)

    # Full Screen option
    folium.plugins.Fullscreen().add_to(map_plot)

    # Layer control for each layers added can be checked & unchecked while selecting
    folium.LayerControl(position='topright',collapsed=True, autoZIndex=True).add_to(map_plot)

    # Saved the outputs
    map_plot.save(file_path)  
    
    finished_tasks()
    
def geospatial_analysis():
    global data, shapefile_low_res, shapefile_high_res, country_name, map_output, file_name, file_path
    
    counter = 0
    my_list = [data, shapefile_low_res, shapefile_high_res, country_name, map_output, file_name, file_path]
    for i in my_list:
        if (i is None):
            counter += 1
            break
        else:
            continue
        
    if counter == 0:
    
        # get the shapefiles
        gdf_low_res = gpd.read_file(shapefile_low_res)
        gdf_high_res = gpd.read_file(shapefile_high_res)

        # isolate the geometries
        country_gdf_low_res = gdf_low_res.loc[gdf_low_res['NAME'] == country_name] # isolate row
        country_geometry_low_res = country_gdf_low_res.geometry.iloc[0] # get multipolygon
        country_geometry_buffer = country_geometry_low_res.buffer(-0.1)
        country_gdf_high_res = gdf_high_res.loc[gdf_high_res['NAME'] == country_name] # isolate row
        country_geometry_high_res = country_gdf_high_res.geometry.iloc[0] # get multipolygon

        # start variables for coordinate lists and counters
        inside = []
        outside_low_res = pd.DataFrame(columns=data.columns)
        outside_high_res = []
        inside_count = {}
        outside_count = {}
        total_count = len(data)

        # check if it is in the low resolution file
        for idx, row in data.iterrows():
            location = Point(row['longitude'], row['latitude'])
            lat_long = (row['latitude'], row['longitude'])
            if location.within(country_geometry_buffer):
                inside.append((row['ID'], row['latitude'], row['longitude']))
                inside_count[lat_long] = inside_count.get(lat_long, 0)+1
            # add it to a temporary df if point is outside of low_res
            else:
                df_temporary = pd.DataFrame(data = [row], columns=outside_low_res.columns)
                outside_low_res = pd.concat([outside_low_res, df_temporary], ignore_index=True)

        # check for remaining points if they are in the high resolution file
        for idx, row in outside_low_res.iterrows():
            location = Point(row['longitude'], row['latitude'])
            lat_long = (row['latitude'], row['longitude'])
            if location.within(country_geometry_high_res):
                inside.append((row['ID'], row['latitude'], row['longitude']))
                inside_count[lat_long] = inside_count.get(lat_long, 0)+1
                outside_low_res.drop(idx, inplace=True)
            # add it to the list of coordinates outside of the zone
            else:             
                outside_high_res.append((row['ID'], row['latitude'], row['longitude']))
                outside_count[lat_long] = outside_count.get(lat_long, 0)+1

        # save some outputs
        invalid_LL_file_path = file_path.replace(f'{file_name}.html', 'invalid_LL.xlsx')
        outside_low_res.to_excel(invalid_LL_file_path, index=False)
        user_reporting_file_path = file_path.replace(f'{file_name}.html', 'user_reporting.txt')
        total = len(inside) + len(outside_high_res)
        percentage = (len(outside_high_res) / total) * 100
        file = open(user_reporting_file_path, "w")
        file.write(f'Total points inside: {len(inside)} \n' 
                   f'Total points outside: {len(outside_high_res)} \n' 
                   f'Total points: {total} \n' 
                   f'Percentage of data outside: {percentage:.1f}% \n' )
        file.close()

        plot_maps(data, country_geometry_high_res, map_output, inside, outside_high_res, inside_count, file_path, country_geometry_low_res)

    else:
        messagebox.showerror(title="empty fields", message="there are empty fields, please go back and ensure all steps have been completed.")
        
def on_closing():
    window.quit()
    window.destroy()

if __name__ == '__main__':
    
    # create a window object
    window=tkinter.Tk()
    window.title('Coordinate Validation Tool')
    window.geometry('448x400')
    
    # Create a canvas that fills the entire window
    canvas = tkinter.Canvas(window)
    canvas.place(relx=0.5, rely=0.5, anchor='center')
    frame = tkinter.Frame(canvas)
    canvas.create_window(3, 0, window=frame)
    def set_scroll_to_top(event):
        canvas.yview_moveto(0)
    canvas.bind("<Map>", set_scroll_to_top)

    # add a title
    label = tkinter.Label(frame, text='This is the Coordinate Validation Tool. Follow the Below Steps').pack(pady=5)

    # add a button to get the user to select the excel file
    label = tkinter.Label(frame, text='    Step 1: Select the excel workbook containing the ID, lat and long coordinates').pack()
    button = tkinter.Button(frame, text='Choose File', command = get_excel_doc).pack(pady=15)
    
    # add a listbox to get user to select country, add scrollbar and confirm button
    label = tkinter.Label(frame, text='Step 2: Select the country, then hit confirm').pack()
    listbox_frame = tkinter.Frame(frame)
    listbox_frame.pack()
    listvar = tkinter.StringVar(value=['Antigua and Barbuda', 'Argentina', 'Bahamas', 'Barbados', 'Belize', 
                                       'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Dominica',
                                       'Dominican Republic', 'Ecuador', 'El Salvador', 'Grenada', 'Guatemala',
                                       'Guyana', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama',
                                       'Paraguay', 'Peru', 'Puerto Rico', 'Saint Kitts and Nevis', 'Saint Lucia',
                                       'Saint Vincent and the Grenadines', 'Suriname', 'Trinidad and Tobago',
                                       'Uruguay', 'Venezuela'])
    listbox = tkinter.Listbox(listbox_frame, listvariable=listvar, height=5)
    listbox.grid(row=0, column=0)
    scrollbar = tkinter.Scrollbar(listbox_frame, orient=tkinter.VERTICAL, command=listbox.yview)
    scrollbar.grid(row=0, column=1, sticky=tkinter.NS)
    button = tkinter.Button(frame, text='Confirm', command=choose_country).pack(pady=15)

    # add a listbox to get user to select output, add scrollbar and confirm button
    label = tkinter.Label(frame, text='Step 3: Select the output map, then hit confirm').pack()
    listbox_frame2 = tkinter.Frame(frame)
    listbox_frame2.pack()
    listvar2 = tkinter.StringVar(listbox_frame2, value= ['Heatmap', 'Heatmap Overlap', 'Point Data Map', 'All'])
    listbox_frame = tkinter.Frame(frame)
    listbox_frame.pack()
    listbox2 = tkinter.Listbox(listbox_frame, listvariable=listvar2, height=4)
    listbox2.pack()
    button = tkinter.Button(frame, text='Confirm', command=choose_output)
    button.pack(pady=15)
    
    # add a button to get the desired file name
    label = tkinter.Label(frame, text='Step 4: Choose the name you want to save the file in').pack()
    textentry_file = tkinter.Entry(frame)
    textentry_file.pack()
    button = tkinter.Button(frame, text='Confirm', command = get_file_name).pack(pady=15)
    
    # start analysis
    label = tkinter.Label(frame, text='Step 5: Start Analysis').pack()
    button = tkinter.Button(frame, text='Go', command = geospatial_analysis).pack()
    label = tkinter.Label(frame, text='').pack(pady=15)

    # Add a scrollbar to the window
    def configure_scroll_region(event):
        canvas.configure(scrollregion=canvas.bbox("all"))
    frame.bind("<Configure>", configure_scroll_region)
    scrollbar = tkinter.Scrollbar(window, orient=tkinter.VERTICAL, command=canvas.yview)
    scrollbar.pack(side=tkinter.RIGHT, fill=tkinter.Y)
    
    canvas.configure(yscrollcommand=scrollbar.set)
    canvas.pack(fill='both', expand=True)
    
    
    window.protocol("WM_DELETE_WINDOW", on_closing)
    window.mainloop()
